In [1]:
from define_object import define_object
from make_reference import make_reference
from exposuresource import ExposureSource
from get_images_for_sn import get_images_for_sn
from list_db_images import db

In [2]:
from astropy.io import fits

In [3]:
import pandas as pd
import config
import logging
import db
config.Config.init('/global/u2/a/alexisa/curveball/curveball_ztf.yaml')

In [4]:
expsource = ExposureSource.get( 'ZTF' )

In [5]:
logger = logging.getLogger("main")
logger.setLevel( logging.DEBUG )

In [6]:
#import csv file with data
dfSN = pd.read_csv('/global/u2/a/alexisa/SN-IIpdata_final.csv')

SN_name = dfSN.Name
SN_ra = dfSN.RA
SN_dec = dfSN.Dec
SN_green = dfSN.band_g
SN_z=dfSN.z
SN_red = dfSN.band_r
SN_i = dfSN.band_i
SN_t = dfSN.t_peak


# 1. Define the object in database with define_object

In [ ]:

#singular object: parameters define_object( ZTFname, ra, dec, peak_t, z, classification=None, confidence=1.0, othernames=None, curdb=None )
define_object('ZTF18aawczad', 194.569496657143, 27.7292541142857, 58269, z=None, classification=None, confidence=1.0, othernames=None, curdb=None )

In [ ]:
#run through multiple objects
for i in range(1,34):
    define_object(SN_name[i], SN_ra[i], SN_dec[i], SN_t[i], z=None, classification=None, confidence=1.0, othernames=None, curdb=None )

# 2. Pull images of the supernova so that a reference can get built 

In [8]:
#get_images_for_sn- the exposure source specifies the type of survey we want this function to pull the data from
#Start off and endoff date are the days you want the lightcurve data to include, typically -30 to 90 is standard 
#However, for SN-IIp specifically, their plateu phase is much longer, so -30 to 270 is a good range to have
#It just depends on the type of SN you have really!

obj = 'ZTF19acchaza'

get_images_for_sn(expsource, obj, ra=None, dec=None, peakt=None,
                       startoff=-30, endoff=270, datadir=None, curdb=None, clobber=True,
                       num_images=None, logger= logger)


In [ ]:
#This is to run through a group of objects
for i in range(0,31):
    get_images_for_sn(expsource, obj=SN_name[i], ra=None, dec=None, peakt=None,
                       startoff=-30, endoff=90, datadir=None, curdb=None, clobber=True,
                       num_images=None, logger= logger)

In [9]:
#if you want to check to see if images were pulled for a specific object
for i in range(20, 35):
    print(i)
    y = db.Image.get_including_point( SN_ra[i], SN_dec[i], band=SN_red[i], expsourceid=expsource.id )
    print(y)

NameError: name 'red_ra' is not defined

# 3. Build the reference for that SN
The band parameter refers to whether you build the reference for the i, red, or green. When you are running this, that will be the only parameter you change once you build all the references in a specific band. So whatever you name those columns in the dataframe, is what you put there. For me that was: SN_red, SN_green, SN_i. 

Then when I run into a reference that won't build, I alter the range I run, so that it skips the bad one. Let's say number 4 the reference could not build, then I would run for i in range(0,4), and then run: for i in range(5,to however many SN there are)

obj = SN_name[i] is whatever you named your dataframe of the SN names. 

Now there was one case, where I had to build a lightcurve that skipped a few days, since there was another SN that occured during this time. You input the gap of days you want the lightcurve model to skip in the secure_references_kwargs={}. 

In [ ]:
#for running through a group of objects, can be changed to just run for one
errorSN = []
for i in range(68,71):
    print(i)
    print(SN_name[i])
    try:
    make_reference(expsource, band = SN_i[i], imagelis=None, imagefilename=None,
                    secure_references_kwargs={},
                    stackname=None, obj= SN_name[i], ra=None, dec=None, peakt=None,
                    datadir=None, outdir=None, workdir=None, searchlocal=False, bgsub=True,
                    curdb=None, logger=logger)
     except Exception as err:
                print('ERROR! REFERENCE CANT BE BUILT: %s' % SN_name[i])
                errorSN.append(SN_name[i])

In [ ]:
#for skipping a gap of days
secure = { "days_before": 472, "days_after": 500 } 
make_reference(expsource, band = SN_i[0], imagelis=None, imagefilename=None,
                    secure_references_kwargs=secure,
                    stackname=None, obj= SN_name[0], ra=None, dec=None, peakt=None,
                    datadir=None, outdir=None, workdir=None, searchlocal=False, bgsub=True, 
                    curdb=None, logger=logger)

In [ ]:
#for running on a single SN 
obj = 'ZTF19acchaza'
make_reference(expsource, band = 'ZTF_r', imagelis=None, imagefilename=None,
                    secure_references_kwargs={},
                    stackname=None, obj= obj, ra=None, dec=None, peakt=None,
                    datadir=None, outdir=None, workdir=None, searchlocal=False, bgsub=True, 
                    curdb=None, logger=logger)


> WARNING: default.sex not found, using internal defaults


> WARNING: default.psfex not found, using internal defaults

/home/curveball/bin/bijaouisky.py:46: RuntimeWarning: divide by zero encountered in log
  val = -( nI * np.log( pI ) ).sum()
/home/curveball/bin/bijaouisky.py:46: RuntimeWarning: invalid value encountered in multiply
  val = -( nI * np.log( pI ) ).sum()
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: overflow encountered in exp
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: invalid value encountered in multiply
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:41: RuntimeWarning: invalid value encountered in true_divide
  pI /= pI.sum()
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: overflow encountered in multiply
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)

> 
----- MissFITS 2.8.0 started on 2023-03-22 at 12:02:01 with 1 thread

> Examining input data...
> Looking for /data/curveball/ZTF/2022-09-05/ztf_20220905472303_000740_zr_io.083.head
> Reading input data...
> 
ztf_20220905472303_000740_zr_io.083.fits      xxx
> 
> All done (in 0.0 s)



> WARNING: default.sex not found, using internal defaults


> WARNING: default.psfex not found, using internal defaults

/home/curveball/bin/bijaouisky.py:46: RuntimeWarning: divide by zero encountered in log
  val = -( nI * np.log( pI ) ).sum()
/home/curveball/bin/bijaouisky.py:46: RuntimeWarning: invalid value encountered in multiply
  val = -( nI * np.log( pI ) ).sum()
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: overflow encountered in exp
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: invalid value encountered in multiply
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:41: RuntimeWarning: invalid value encountered in true_divide
  pI /= pI.sum()
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: overflow encountered in multiply
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)

> 
----- MissFITS 2.8.0 started on 2023-03-22 at 12:04:22 with 1 thread

> Examining input data...
> Looking for /data/curveball/ZTF/2022-09-24/ztf_20220924455660_000740_zr_io.083.head
> Reading input data...
> 
ztf_20220924455660_000740_zr_io.083.fits      xxx
> 
> All done (in 0.0 s)



> WARNING: default.sex not found, using internal defaults


> WARNING: default.psfex not found, using internal defaults

/home/curveball/bin/bijaouisky.py:46: RuntimeWarning: divide by zero encountered in log
  val = -( nI * np.log( pI ) ).sum()
/home/curveball/bin/bijaouisky.py:46: RuntimeWarning: invalid value encountered in multiply
  val = -( nI * np.log( pI ) ).sum()
/home/curveball/bin/bijaouisky.py:41: RuntimeWarning: invalid value encountered in true_divide
  pI /= pI.sum()
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: overflow encountered in multiply
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: invalid value encountered in multiply
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: overflow encountered in exp
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)

...


> WARNING: default.sex not found, using internal defaults


> WARNING: default.psfex not found, using internal defaults

/home/curveball/bin/bijaouisky.py:46: RuntimeWarning: divide by zero encountered in log
  val = -( nI * np.log( pI ) ).sum()
/home/curveball/bin/bijaouisky.py:46: RuntimeWarning: invalid value encountered in multiply
  val = -( nI * np.log( pI ) ).sum()
/home/curveball/bin/bijaouisky.py:41: RuntimeWarning: invalid value encountered in true_divide
  pI /= pI.sum()
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: overflow encountered in multiply
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: invalid value encountered in multiply
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: overflow encountered in exp
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)

> 
----- MissFITS 2.8.0 started on 2023-03-22 at 12:09:11 with 1 thread

> Examining input data...
> Looking for /data/curveball/ZTF/2022-10-06/ztf_20221006371250_000740_zr_io.083.head
> Reading input data...
> 
ztf_20221006371250_000740_zr_io.083.fits      xxx
> 
> All done (in 0.0 s)



> WARNING: default.sex not found, using internal defaults


> WARNING: default.psfex not found, using internal defaults

/home/curveball/bin/bijaouisky.py:46: RuntimeWarning: divide by zero encountered in log
  val = -( nI * np.log( pI ) ).sum()
/home/curveball/bin/bijaouisky.py:46: RuntimeWarning: invalid value encountered in multiply
  val = -( nI * np.log( pI ) ).sum()
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: overflow encountered in exp
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: invalid value encountered in multiply
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:40: RuntimeWarning: overflow encountered in multiply
  pI = 1/a * np.exp(σ**2 / (2*a**2) ) * np.exp(-(I-s)/a) * 0.5 * erfc( ( σ/a - (I-s)/σ ) / sqrt2 )
/home/curveball/bin/bijaouisky.py:41: RuntimeWarning: invalid valu

> 
----- MissFITS 2.8.0 started on 2023-03-22 at 12:11:23 with 1 thread

> Examining input data...
> Looking for /data/curveball/ZTF/2022-10-08/ztf_20221008428553_000740_zr_io.083.head
> Reading input data...
> 
ztf_20221008428553_000740_zr_io.083.fits      xxx
> 
> All done (in 0.0 s)



> WARNING: default.sex not found, using internal defaults


> WARNING: default.psfex not found, using internal defaults



# Some errors you might get: 

empty of corrupt FITS file 
list index out of range --can't do much about this

No images found for filter zi, No references found for filter zi --object either does not have a reference image, or it just can't build, so dont worry about it 

RuntimeError: Stack stack_ztf_20201205497674_000672_zi_io.154 is already in the database with different members!  --Made a stack from the different images but in the same file so it will bring up this error, try to delete the stack that already exists through terminal 

(ValueError: Filter ZTF g (from header of /data/curveball/ZTF/2018-03-25/ztf_20180325298773_000789_zg_io.062.fits) is unknown. --this means the header has ZTF g instead of ZTF_g, so try to change it yourself in the file

Exception: psfex failed for ztf_20200427402512_000677_zg_io.142.cat --means there is something fishy about the image, inspect fits file

Overall the two errors that are within your control to fix is the error about stacking and the error about filter ZTF g. As I go through the objects, I mark which ones gave me errors, and I rerun them a week or two later. Typically, some of the references do work when I try again, which means it could be something due to Cori or Perlmutter being finnicky during the first time you're using it. 

# 4. After the references are built, you need to create the jobs that build the lightcurves for each SN on the curveball database. Each object will have three jobs you will have to run on NERSC (since you have three filters, g, r, and i). I have created a separate .ipynb for how to do this

In [7]:
#for running through a group of objects, can be changed to just run for one
errorSN = []
for i in range(0,72):
    print(i)
    print(SN_name[i])
    try:
        make_reference(expsource, band = SN_green[i], imagelis=None, imagefilename=None,
                    secure_references_kwargs={},
                    stackname=None, obj= SN_name[i], ra=None, dec=None, peakt=None,
                    datadir=None, outdir=None, workdir=None, searchlocal=False, bgsub=True,
                    curdb=None, logger=logger)
    except Exception as err:
            print('ERROR! REFERENCE CANT BE BUILT: %s' % SN_name[i])
            errorSN.append(SN_name[i])

0
ZTF18abgqvwv
1
ZTF18adbacau
2
ZTF19aafndot
3
ZTF19aamggnw
4
ZTF19aapygmq
5
ZTF19aarjswp
6
ZTF19aauqwna



> WARNING: default.sex not found, using internal defaults


> WARNING: default.psfex not found, using internal defaults

psfex failed with fwhmmax=10.0, bumping it (or quitting)
Traceback (most recent call last):
  File "/curveball/bin/processing.py", line 385, in cat_psf_sky
    res = subprocess.run( com.split(), shell=False, timeout=psftimeout )
  File "/usr/lib/python3.9/subprocess.py", line 507, in run
    stdout, stderr = process.communicate(input, timeout=timeout)
  File "/usr/lib/python3.9/subprocess.py", line 1134, in communicate
    stdout, stderr = self._communicate(input, endtime, timeout)
  File "/usr/lib/python3.9/subprocess.py", line 2007, in _communicate
    self.wait(timeout=self._remaining_time(endtime))
  File "/usr/lib/python3.9/subprocess.py", line 1189, in wait
    return self._wait(timeout=timeout)
  File "/usr/lib/python3.9/subprocess.py", line 1911, in _wait
    raise TimeoutExpired(self.args, timeout)
subprocess.TimeoutExpired: Command '['psfex', '/data/curveb

ERROR! REFERENCE CANT BE BUILT: ZTF19aauqwna
7
ZTF19aavbkly
8
ZTF19aawgxdn
9
ZTF19aaxqwjx
10
ZTF19abajxet
11
ZTF19abqgtqo
12
ZTF19abueupg
13
ZTF19abuzinv
14
ZTF19acytcsg
15
ZTF20acfkzcg
16
ZTF21aaabwem


No images found for filter zg
No references found for filter zg


ERROR! REFERENCE CANT BE BUILT: ZTF21aaabwem
17
ZTF22aadpuwe
18
ZTF18abcpmwh
19
ZTF18abcptmt
20
ZTF18abfzixe
21
ZTF18abwlsoi
22
ZTF20acrinvz
23
ZTF18aarpttw
24
ZTF18aaszvfn
25
ZTF18aatlfus
26
ZTF18aatyqds
27
ZTF18aaxkqgy
28
ZTF18abbpyvk
29
ZTF18abckutn
30
ZTF18abdbysy
31
ZTF18abdfwcy
32
ZTF18abdjmfh
33
ZTF18abdrbqp
34
ZTF18abgladq
35
ZTF18abrlljc
36
ZTF18achtnvk
37
ZTF18acyybvg
38
ZTF19aallimd


...

No images found for filter zg
No references found for filter zg


ERROR! REFERENCE CANT BE BUILT: ZTF19abrtjsf
46
ZTF19abtslqm
47
ZTF19abvbrve


No images found for filter zg
No references found for filter zg


ERROR! REFERENCE CANT BE BUILT: ZTF19abvbrve
48
ZTF19abwamby
49
ZTF19acbhvgi
50
ZTF19acbrzzr
51
ZTF19accbeju
52
ZTF19acftfav
53
ZTF19acgbkzr
54
ZTF19acgzzea
55
ZTF19acifduk
56
ZTF19acignlo
57
ZTF19aclobbu
58
ZTF20aarbvub
59
ZTF20abwftit
60
ZTF20abxmwwd
61
ZTF21aaagypx
62
ZTF21aajgdeu
63
ZTF21acceboj
64
ZTF21aciobmz
65
ZTF22aajivyw
66
ZTF22aapqaqe
67
ZTF22abhsxph
68
ZTF19aazudta
69
ZTF19aaydtur
70
ZTF18aahfqbc
71


KeyError: 71

In [8]:
print (errorSN)

['ZTF19aauqwna', 'ZTF21aaabwem', 'ZTF19aallimd', 'ZTF19aarykkb', 'ZTF19abiahko', 'ZTF19abrtjsf', 'ZTF19abvbrve']
